In [15]:
import os
from minio import Minio
import requests

In [ ]:
client = Minio("minio-service.kubeflow.svc.cluster.local:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False,           
)

In [ ]:
type(client)

In [ ]:
bucket_name = "newtiledb"

In [ ]:
def upload_files(bucket_name, file_location, client):
    found = False  # Initialize 'found' before the try block
    try:
        found = client.bucket_exists(bucket_name)
    except Exception as e:
        print("error trying to search for MinIO Bucket:", e)
        return  # Return early since we cannot proceed without knowing if the bucket exists

    if not found:
        try:
            client.make_bucket(bucket_name)
            print("Created bucket", bucket_name)
        except Exception as e:
            print("Failed to create bucket:", e)
            return  # Return early since we cannot proceed if the bucket cannot be created
    else:
        print("Bucket", bucket_name, "exists, we won't attempt to create one")
        
    # Ensure the directory exists
    if not os.path.isdir(file_location):
        print(f"The directory {file_location} does not exist.")
        return

    # Iterate through all files in the directory
    for file_name in os.listdir(file_location):
        # Construct the full file path
        source_file = os.path.join(file_location, file_name)
        # Check if it's a file and not a directory
        if os.path.isfile(source_file):
            try:
                # Upload the file
                client.fput_object(bucket_name, file_name, source_file)
                print(f"Successfully uploaded {file_name} to bucket {bucket_name}.")
            except Exception as e:
                print(f"Failed to upload {file_name}: {e}")


In [ ]:
upload_files(bucket_name,"documentation",client)

In [25]:
data = {
  "instances": [{
    "input": "what are vectors",
    "num_docs": 4  # number of documents to retrieve
  }]
}

URL = "http://vectorstore-predictor.christensenc3526.svc.cluster.local/v1/models/vectorstore:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
print(response.json())
print(response.text)

<Response [200]>
{'predictions': ['Vector databases are specialized storage systems designed to efficiently store and query high-dimensional vector data, often used in machine learning, artificial intelligence, and similarity search applications. This section introduces the concept of vector databases, their importance in enabling fast and scalable vector similarity searches, and typical use cases such as image and text retrieval, and recommendation systems.\n2. Getting Started with Vector Databases', 'Vector Databases Documentation\n1. Introduction to Vector Databases', 'This guide provides a step-by-step introduction to working with vector databases, including setting up your first vector database, inserting and indexing vector data, and performing similarity searches. Through practical examples, learn the basics of vector storage, querying techniques, and how to integrate vector databases into your data pipeline.\n3. Vector Database Architecture and Indexing Techniques', 'Understand

In [44]:
URL = URL = "http://llm-transformer.christensenc3526.svc.cluster.local/v1/models/llm:predict"

In [ ]:
data = {
  "instances": [{
      "system": "You are an AI assistant. You will be given a task. You must generate a detailed answer.",
      "instruction": "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.",
      "input": "Is TileDB a good vector store?",
      "max_tokens": 50,
      "top_k": 100,
      "top_p": 0.4,
      "num_docs": 1,
      "temperature": 0.2
  }]
}
response = requests.post(URL, json=data,verify=False)

In [ ]:
print(response)
#print(response.json())
print(response.text)